In [1]:
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
# !tar xf spark-2.4.4-bin-hadoop2.7.tgz
# !pip install -q findspark

# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

import findspark
findspark.init("../dataset/spark-2.4.4-bin-hadoop2.7")# SPARK_HOME

from pyspark import SparkConf, SparkContext

In [2]:
conf = SparkConf().setMaster("local").setAppName("MovieLensInsights")
sc = SparkContext(conf = conf)

In [3]:
linesRDD = sc.textFile('../dataset/ml-latest-small/ratings.csv')
header = linesRDD.first() #extract header
linesRDD = linesRDD.filter(lambda x: x != header) #filter out header
ratingsRDD = linesRDD.map(lambda x: x.encode("ascii", "ignore").split(',')[:-1]) #removing the timestamp column

In [4]:
print(ratingsRDD.collect()[:10])

[['1', '1', '4.0'], ['1', '3', '4.0'], ['1', '6', '4.0'], ['1', '47', '5.0'], ['1', '50', '5.0'], ['1', '70', '3.0'], ['1', '101', '5.0'], ['1', '110', '4.0'], ['1', '151', '5.0'], ['1', '157', '5.0']]


In [5]:
userRatingsRDD = ratingsRDD.map(lambda x: (x[0],(float(x[2]), 1)))\
                .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
                .map(lambda x: ("u_" + x[0], x[1][0] / x[1][1]))\
                .sortBy(lambda x: -x[1])
print(userRatingsRDD.collect())

[('u_53', 5.0), ('u_251', 4.869565217391305), ('u_515', 4.846153846153846), ('u_25', 4.8076923076923075), ('u_30', 4.735294117647059), ('u_523', 4.693333333333333), ('u_348', 4.672727272727273), ('u_171', 4.634146341463414), ('u_452', 4.556930693069307), ('u_43', 4.552631578947368), ('u_371', 4.548780487804878), ('u_122', 4.546232876712328), ('u_441', 4.522222222222222), ('u_400', 4.511627906976744), ('u_52', 4.476923076923077), ('u_538', 4.472972972972973), ('u_168', 4.462765957446808), ('u_417', 4.462686567164179), ('u_543', 4.453947368421052), ('u_106', 4.4393939393939394), ('u_319', 4.428571428571429), ('u_601', 4.425742574257426), ('u_413', 4.410714285714286), ('u_475', 4.409677419354839), ('u_188', 4.395833333333333), ('u_12', 4.390625), ('u_276', 4.390243902439025), ('u_154', 4.382352941176471), ('u_533', 4.381578947368421), ('u_581', 4.375), ('u_69', 4.369565217391305), ('u_1', 4.366379310344827), ('u_586', 4.365384615384615), ('u_544', 4.363636363636363), ('u_128', 4.363636363

In [6]:
movieRatingsRDD = ratingsRDD.map(lambda x: (x[1],(float(x[2]), 1)))\
                .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
                .map(lambda x: ("m_" + x[0], x[1][0] / x[1][1]))\
                .sortBy(lambda x: -x[1])
print(movieRatingsRDD.collect())

[('m_5745', 5.0), ('m_172793', 5.0), ('m_27373', 5.0), ('m_99', 5.0), ('m_74226', 5.0), ('m_6818', 5.0), ('m_147196', 5.0), ('m_136850', 5.0), ('m_3687', 5.0), ('m_2512', 5.0), ('m_3531', 5.0), ('m_134095', 5.0), ('m_102084', 5.0), ('m_1151', 5.0), ('m_117531', 5.0), ('m_96430', 5.0), ('m_140627', 5.0), ('m_146662', 5.0), ('m_141718', 5.0), ('m_187717', 5.0), ('m_136834', 5.0), ('m_8738', 5.0), ('m_147300', 5.0), ('m_4180', 5.0), ('m_96608', 5.0), ('m_64499', 5.0), ('m_45503', 5.0), ('m_115727', 5.0), ('m_6086', 5.0), ('m_172875', 5.0), ('m_175397', 5.0), ('m_109687', 5.0), ('m_86721', 5.0), ('m_4788', 5.0), ('m_157775', 5.0), ('m_179135', 5.0), ('m_179133', 5.0), ('m_147286', 5.0), ('m_47736', 5.0), ('m_136341', 5.0), ('m_495', 5.0), ('m_496', 5.0), ('m_92494', 5.0), ('m_3939', 5.0), ('m_107951', 5.0), ('m_130970', 5.0), ('m_130978', 5.0), ('m_5416', 5.0), ('m_131610', 5.0), ('m_91355', 5.0), ('m_67618', 5.0), ('m_140133', 5.0), ('m_3951', 5.0), ('m_4454', 5.0), ('m_163072', 5.0), ('m

In [7]:
import csv

datafile = open('../dataset/ml-latest-small/movies.csv', 'r')
myreader = csv.reader(datafile)

movieGenres = {}
for row in myreader:
    movieGenres[row[0]] = row[2].split('|')

In [8]:
print(movieGenres)

{'110553': ['Action', 'Sci-Fi', 'IMAX'], '60141': ['Children', 'Comedy'], '172793': ['Adventure', 'Animation', 'Children', 'Fantasy'], '101765': ['Adventure', 'Comedy', 'Romance'], '188751': ['Comedy', 'Romance'], '114074': ['Drama'], '55292': ['Comedy'], '5988': ['Drama'], '5989': ['Crime', 'Drama'], '55294': ['Comedy', 'Crime', 'Drama'], '5980': ['Horror', 'Mystery', 'Thriller'], '5986': ['Drama'], '5984': ['Drama', 'Romance'], '103543': ['Comedy', 'Drama'], '87960': ['Documentary'], '165103': ['Comedy'], '102819': ['Drama'], '43871': ['Crime', 'Drama', 'Thriller'], '6796': ['Crime', 'Drama'], '6797': ['Crime', 'Drama'], '6794': ['Children', 'Comedy'], '27373': ['Drama'], '6793': ['Children', 'Comedy', 'Drama'], '6790': ['Drama', 'Fantasy', 'Sci-Fi'], '6791': ['Drama'], '36708': ['Adventure', 'Animation', 'Comedy'], '6798': ['Children', 'Comedy', 'Crime', 'Musical'], '166558': ['Action', 'Horror'], '142602': ['Comedy'], '75947': ['Documentary'], '99992': ['Crime', 'Drama', 'Thriller'

In [9]:
sampleRating = sc.parallelize([[1, 4.0], [1, 2], [2, 1.5]])
sampleGenres = dict()
sampleGenres[1] = ['Action', 'Drama']
sampleGenres[2] = ['sci-fi', 'Action']

genreRatingsRDD = sampleRating.flatMap(lambda x: ((genre, (float(x[1]), 1)) for genre in sampleGenres[x[0]]))\
                .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
                .map(lambda x: ("" + x[0], x[1][0] / x[1][1]))\
                .sortBy(lambda x: -x[1])
print(genreRatingsRDD.collect())

[('Drama', 3.0), ('Action', 2.5), ('sci-fi', 1.5)]


In [10]:
genreRatingsRDD = ratingsRDD.flatMap(lambda x: ((genre, (float(x[2]), 1)) for genre in movieGenres[x[1]]))\
                .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
                .map(lambda x: (x[0], x[1][0] / x[1][1]))\
                .sortBy(lambda x: -x[1])
print(genreRatingsRDD.collect())

[('Film-Noir', 3.920114942528736), ('War', 3.8082938876312), ('Documentary', 3.797785069729286), ('Crime', 3.658293867274144), ('Drama', 3.6561844113718758), ('Mystery', 3.632460255407871), ('Animation', 3.6299370349170004), ('IMAX', 3.618335343787696), ('Western', 3.583937823834197), ('Musical', 3.5636781053649105), ('Adventure', 3.5086089151939075), ('Romance', 3.5065107040388437), ('Thriller', 3.4937055799183425), ('Fantasy', 3.4910005070136894), ('(no genres listed)', 3.4893617021276597), ('Sci-Fi', 3.455721162210752), ('Action', 3.447984331646809), ('Children', 3.412956125108601), ('Comedy', 3.3847207640898267), ('Horror', 3.258195034974626)]
